# House Prices: Advanced Regression Techniques - 初心者向けガイド

このノートブックでは、House Pricesコンペに取り組みながら、**回帰問題**の機械学習を学びます。

## 📚 このコンペについて

**問題の種類**: 回帰（Regression）
- **回帰とは**: 連続した数値を予測する問題です
- このコンペでは、住宅の特徴（面積、部屋数、立地など）から**住宅価格**を予測します
- 分類問題（0か1）とは異なり、**連続した値**（例: $150,000, $200,000など）を予測します

**評価指標**: RMSE（Root Mean Squared Error / 平均二乗誤差の平方根）
- **RMSEとは**: 予測値と実際の値の差を測る指標です
- 値が小さいほど良い（完璧な予測なら0）
- 単位は目的変数と同じ（この場合、ドル）

**データ形式**: 表形式データ（Tabular Data）
- CSVファイルのような表形式のデータ
- 各行が1つの住宅、各列が特徴（面積、部屋数など）を表します

## 🔄 分類問題との違い

| 項目 | 分類問題（Titanicなど） | 回帰問題（House Prices） |
|------|----------------------|----------------------|
| 予測する値 | カテゴリ（0か1） | 連続した数値（価格など） |
| 評価指標 | Log Loss, Accuracy | RMSE, MAE |
| モデル | 分類器 | 回帰器 |
| 例 | 生存したかどうか | 住宅価格はいくらか |


## 🎯 回帰問題の基本用語

### 重要な用語集

1. **回帰（Regression）**
   - 連続した数値を予測する問題
   - 例: 価格、温度、売上など

2. **RMSE（Root Mean Squared Error）**
   - 予測値と実際の値の差を測る指標
   - 式: √(平均((予測値 - 実際の値)²))
   - 外れ値（大きく外れた予測）に敏感

3. **MAE（Mean Absolute Error）**
   - 予測値と実際の値の差の絶対値の平均
   - 外れ値に比較的強い

4. **特徴量エンジニアリング**
   - このコンペでは特に重要！
   - 既存の特徴量から新しい特徴量を作る
   - 例: 総面積 = 1階の面積 + 2階の面積


## 📊 全体の流れ（ワークフロー）

```
1. ライブラリのインポート
   ↓
2. データの読み込み
   ↓
3. データ探索（EDA）
   - 目的変数（価格）の分布を確認
   - 特徴量の分布、欠損値を確認
   ↓
4. 特徴量エンジニアリング（重要！）
   - 新しい特徴量の作成
   - カテゴリ変数の処理
   ↓
5. データ前処理
   - 欠損値の処理
   - スケーリング
   ↓
6. モデルの訓練
   - 線形回帰、ランダムフォレスト、勾配ブースティングなど
   ↓
7. 予測の生成
   - テストデータに対して予測を実行
   ↓
8. 提出ファイルの作成
   - Kaggle提出用のCSVファイルを作成
```


## ステップ1: ライブラリのインポート

回帰問題用のライブラリをインポートします。


In [ ]:
# 必要なライブラリをインポート
import pandas as pd  # データ操作
import numpy as np   # 数値計算
import matplotlib.pyplot as plt  # グラフ描画
import seaborn as sns  # より見やすいグラフ
import warnings  # 警告を非表示にする

# 機械学習関連のライブラリ（回帰用）
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error

# EDA自動化ライブラリ（オプション）
try:
    from ydata_profiling import ProfileReport
    EDA_AVAILABLE = True
    print("✅ ydata-profiling が利用可能です")
except ImportError:
    EDA_AVAILABLE = False
    print("⚠️ ydata-profiling がインストールされていません")

# 警告を非表示にする
warnings.filterwarnings('ignore')

# グラフのスタイル設定
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ ライブラリのインポートが完了しました！")


## ステップ2: データの読み込み

### データのダウンロード方法

1. **Kaggle APIを使う場合**:
   ```bash
   kaggle competitions download -c house-prices-advanced-regression-techniques -p input/
   cd input/
   unzip house-prices-advanced-regression-techniques.zip
   ```

2. **KaggleのWebサイトから手動でダウンロード**:
   - コンペページの「Data」タブからダウンロード
   - `input/house-prices-advanced-regression-techniques/` フォルダに保存

### データファイルについて

- `train.csv`: 訓練データ（特徴量 + 住宅価格）
- `test.csv`: テストデータ（特徴量のみ）
- `sample_submission.csv`: 提出ファイルの形式例
- `data_description.txt`: 各特徴量の説明（重要！）


In [ ]:
# データの読み込み
# 注意: Docker環境では ../input/ のパスを使用します
# 整理後のディレクトリ構造: input/house-prices-advanced-regression-techniques/

try:
    # まずローカル環境（Docker）のパスを試す（整理後のディレクトリ構造）
    train_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
    test_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
    print("✅ ローカル環境（整理後のディレクトリ）からデータを読み込みました")
except FileNotFoundError:
    try:
        # Kaggle環境の場合
        train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
        test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
        print("✅ Kaggle環境からデータを読み込みました")
    except FileNotFoundError:
        # 別のパスを試す
        train_df = pd.read_csv('input/house-prices-advanced-regression-techniques/train.csv')
        test_df = pd.read_csv('input/house-prices-advanced-regression-techniques/test.csv')
        print("✅ 別のパスからデータを読み込みました")

# データの基本情報を確認
print(f"
📊 訓練データのサイズ: {train_df.shape}")
print(f"📊 テストデータのサイズ: {test_df.shape}")
print(f"
訓練データの最初の5行:")
train_df.head()


## ステップ3: データ探索（EDA）

回帰問題では、**目的変数（価格）の分布**を確認することが特に重要です。
価格が正規分布に近いか、外れ値があるかなどを確認します。


In [ ]:
# 目的変数（SalePrice）の確認（最も簡単な実装）
target_col = 'SalePrice'

if target_col in train_df.columns:
    # 基本統計
    print(train_df[target_col].describe())
    print(f"\n💡 歪度: {train_df[target_col].skew():.2f}")
    
    # pandasのplotメソッドで1行で可視化
    train_df[target_col].hist(bins=50, figsize=(10, 5), edgecolor='black')
    plt.title('住宅価格の分布')
    plt.xlabel('価格（ドル）')
    plt.ylabel('件数')
    plt.show()
else:
    print("⚠️ 目的変数 'SalePrice' が見つかりませんでした")


In [ ]:
### 簡潔なEDA: ライブラリを使った一括可視化

以下のコードで、主要なEDAを一度に実行できます。


In [ ]:
# 簡潔なEDA: 基本情報と欠損値（最も簡単な実装）
# pandasのinfo()で基本情報を一括表示
train_df.info()

# 欠損値の多い特徴量（上位10個）を可視化
missing = train_df.isnull().sum().sort_values(ascending=False).head(10)
if missing.sum() > 0:
    missing[missing > 0].plot(kind='barh', figsize=(8, 6))
    plt.title('欠損値の多い特徴量（上位10個）')
    plt.show()


In [ ]:
# 目的変数と重要な特徴量の一括可視化（コンパクト版）
target_col = 'SalePrice'

if target_col in train_df.columns:
    # 数値特徴量を取得（Idと目的変数を除外）
    numeric_cols = [c for c in train_df.select_dtypes(include=[np.number]).columns 
                    if c not in ['Id', target_col]]
    
    # 相関が高い特徴量を取得（上位4個）
    top_features = train_df[numeric_cols + [target_col]].corr()[target_col].abs().sort_values(ascending=False).head(5).index.tolist()
    top_features = [f for f in top_features if f != target_col][:4]
    
    # 可視化（2行2列）
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    # 目的変数の分布
    axes[0].hist(train_df[target_col], bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_title(f'{target_col}の分布')
    
    # 上位3個の特徴量と目的変数の散布図
    corr = train_df[numeric_cols + [target_col]].corr()[target_col]
    for i, feature in enumerate(top_features[:3], 1):
        axes[i].scatter(train_df[feature], train_df[target_col], alpha=0.3)
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel(target_col)
        axes[i].set_title(f'{feature} (相関: {corr[feature]:.2f})')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"💡 相関が高い特徴量: {top_features[:3]}")


In [ ]:
# 相関行列と外れ値の一括可視化
if target_col in train_df.columns:
    # 数値特徴量を取得（まだ定義されていない場合）
    if 'numeric_cols' not in locals():
        numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
        if 'Id' in numeric_cols:
            numeric_cols.remove('Id')
        if target_col in numeric_cols:
            numeric_cols.remove(target_col)
    
    if len(numeric_cols) > 0:
        # 相関が高い特徴量を選択
        correlations = train_df[numeric_cols + [target_col]].corr()[target_col].abs().sort_values(ascending=False)
        top_features = correlations.head(10).index.tolist()
        if target_col in top_features:
            top_features.remove(target_col)
        
        # 2つの図を並べて表示
        fig, axes = plt.subplots(1, 2, figsize=(18, 7))
        
        # 1. 相関行列（ヒートマップ）
        corr_matrix = train_df[top_features[:8] + [target_col]].corr()
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                    square=True, linewidths=0.5, ax=axes[0], cbar_kws={"shrink": 0.8})
        axes[0].set_title('特徴量間の相関行列（重要な特徴量のみ）')
        
        # 2. 目的変数の箱ひげ図（外れ値検出）
        axes[1].boxplot(train_df[target_col])
        axes[1].set_title(f'{target_col}の箱ひげ図（外れ値検出）')
        axes[1].set_ylabel('価格（ドル）')
        
        plt.tight_layout()
        plt.show()
        
        # 外れ値の数だけ表示
        Q1 = train_df[target_col].quantile(0.25)
        Q3 = train_df[target_col].quantile(0.75)
        IQR = Q3 - Q1
        outliers_count = len(train_df[(train_df[target_col] < Q1 - 1.5*IQR) | 
                                       (train_df[target_col] > Q3 + 1.5*IQR)])
        print(f"💡 外れ値の数: {outliers_count} ({outliers_count/len(train_df)*100:.1f}%)")


In [ ]:
# カテゴリ特徴量と目的変数の関係（簡潔版）
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()

if target_col in train_df.columns and len(categorical_cols) > 0:
    # 価格差が大きいカテゴリ特徴量を自動で見つける
    cat_price_ranges = {}
    for col in categorical_cols[:5]:  # 最初の5個を分析
        price_range = train_df.groupby(col)[target_col].mean().max() - train_df.groupby(col)[target_col].mean().min()
        cat_price_ranges[col] = price_range
    
    if len(cat_price_ranges) > 0:
        best_cat = max(cat_price_ranges, key=cat_price_ranges.get)
        
        # 可視化
        plt.figure(figsize=(12, 6))
        train_df.boxplot(column=target_col, by=best_cat, ax=plt.gca())
        plt.title(f'{best_cat} と 住宅価格の関係')
        plt.suptitle('')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
        
        print(f"💡 価格差が最も大きいカテゴリ特徴量: {best_cat}")
        print(f"   価格差: ${cat_price_ranges[best_cat]:,.0f}")


### オプション: ydata-profilingを使った自動EDA（最も簡単）

1行のコードで、包括的なEDAレポートを自動生成できます。


In [ ]:
# オプション: ydata-profilingで自動EDAレポート生成（最も簡単）
# 1行のコードで包括的なEDAレポートを生成

if EDA_AVAILABLE:
    print("=" * 60)
    print("📊 自動EDAレポート生成中...")
    print("=" * 60)
    print("⚠️ データが大きい場合、数分かかる可能性があります")
    
    # サンプルデータで実行（全データは時間がかかるため）
    sample_size = min(1000, len(train_df))
    train_sample = train_df.sample(n=sample_size, random_state=42)
    
    print(f"\nサンプルデータ（{sample_size}行）でレポートを生成します...")
    print("   全データで実行する場合は、sample_sizeを増やしてください")
    
    # レポート生成
    profile = ProfileReport(
        train_sample, 
        title="House Prices - EDAレポート",
        minimal=True  # 高速化のため簡易モード
    )
    
    # HTMLレポートを保存
    profile.to_file("../eda_report_house_prices.html")
    print("✅ レポートを生成しました: ../eda_report_house_prices.html")
    print("   ブラウザで開いて確認してください")
    
    # ノートブック内で表示（オプション）
    # profile.to_notebook_iframe()
else:
    print("⚠️ ydata-profilingがインストールされていないため、スキップします")
    print("   インストール: pip install ydata-profiling")


### 詳細なEDA: 数値特徴量とカテゴリ特徴量の分析

データをより深く理解するための分析を行います。


In [ ]:
# 数値特徴量とカテゴリ特徴量を分離
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()

# ID列と目的変数を除外
if 'Id' in numeric_cols:
    numeric_cols.remove('Id')
if 'SalePrice' in numeric_cols:
    numeric_cols.remove('SalePrice')

print("=" * 60)
print("📊 特徴量の分類")
print("=" * 60)
print(f"数値特徴量: {len(numeric_cols)}個")
print(f"カテゴリ特徴量: {len(categorical_cols)}個")
print(f"合計: {len(numeric_cols) + len(categorical_cols)}個")


In [ ]:
# 数値特徴量と目的変数（SalePrice）の相関を確認
print("=" * 60)
print("📊 数値特徴量と目的変数の相関（上位20個）")
print("=" * 60)

if 'SalePrice' in train_df.columns and len(numeric_cols) > 0:
    # 相関を計算
    correlations = train_df[numeric_cols + ['SalePrice']].corr()['SalePrice'].sort_values(ascending=False)
    correlations = correlations.drop('SalePrice')  # 自分自身との相関を除外
    
    # 上位20個を表示
    top_correlations = correlations.head(20)
    print("\n正の相関が高い特徴量（上位10個）:")
    print(top_correlations.head(10))
    
    print("\n負の相関が高い特徴量（下位10個）:")
    print(correlations.tail(10))
    
    # 可視化
    plt.figure(figsize=(10, 12))
    top_correlations.plot(kind='barh')
    plt.title('数値特徴量と目的変数の相関（上位20個）')
    plt.xlabel('相関係数')
    plt.tight_layout()
    plt.show()
    
    # 相関の強い特徴量を保存（後で使うため）
    high_corr_features = correlations[abs(correlations) > 0.3].index.tolist()
    print(f"\n💡 相関が強い特徴量（|相関| > 0.3）: {len(high_corr_features)}個")
    print(f"   これらの特徴量は価格予測に重要です")
else:
    print("⚠️ 目的変数または数値特徴量が見つかりませんでした")


In [ ]:
# カテゴリ特徴量と目的変数の関係を確認
print("=" * 60)
print("📊 カテゴリ特徴量と目的変数の関係")
print("=" * 60)

if 'SalePrice' in train_df.columns and len(categorical_cols) > 0:
    # 各カテゴリ特徴量の統計を計算
    cat_stats = []
    
    for col in categorical_cols[:10]:  # 最初の10個を分析
        stats = train_df.groupby(col)['SalePrice'].agg(['count', 'mean', 'std']).sort_values('mean')
        stats['cv'] = stats['std'] / stats['mean']  # 変動係数
        
        cat_stats.append({
            '特徴量': col,
            'カテゴリ数': train_df[col].nunique(),
            '平均価格の範囲': stats['mean'].max() - stats['mean'].min(),
            '最小平均価格': stats['mean'].min(),
            '最大平均価格': stats['mean'].max()
        })
    
    cat_stats_df = pd.DataFrame(cat_stats)
    print("\nカテゴリ特徴量の統計（最初の10個）:")
    print(cat_stats_df)
    
    # 最も価格差が大きいカテゴリ特徴量を可視化
    if len(cat_stats_df) > 0:
        best_cat = cat_stats_df.loc[cat_stats_df['平均価格の範囲'].idxmax(), '特徴量']
        print(f"\n💡 価格差が最も大きいカテゴリ特徴量: {best_cat}")
        
        # 可視化
        plt.figure(figsize=(12, 6))
        train_df.boxplot(column='SalePrice', by=best_cat, ax=plt.gca())
        plt.title(f'{best_cat} と 住宅価格の関係')
        plt.suptitle('')  # デフォルトのタイトルを削除
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ 目的変数またはカテゴリ特徴量が見つかりませんでした")


### オプション: pandas-profilingを使った自動EDA（最も簡単）

1行のコードで、包括的なEDAレポートを自動生成できます。


In [ ]:
# オプション: ydata-profilingで自動EDAレポート生成（最も簡単）
# 1行のコードで包括的なEDAレポートを生成

if EDA_AVAILABLE:
    print("=" * 60)
    print("📊 自動EDAレポート生成中...")
    print("=" * 60)
    print("⚠️ データが大きい場合、数分かかる可能性があります")
    
    # サンプルデータで実行（全データは時間がかかるため）
    sample_size = min(1000, len(train_df))
    train_sample = train_df.sample(n=sample_size, random_state=42)
    
    print(f"\nサンプルデータ（{sample_size}行）でレポートを生成します...")
    print("   全データで実行する場合は、sample_sizeを増やしてください")
    
    # レポート生成
    profile = ProfileReport(
        train_sample, 
        title="House Prices - EDAレポート",
        minimal=True  # 高速化のため簡易モード
    )
    
    # HTMLレポートを保存
    profile.to_file("../eda_report_house_prices.html")
    print("✅ レポートを生成しました: ../eda_report_house_prices.html")
    print("   ブラウザで開いて確認してください")
    
    # ノートブック内で表示（オプション）
    # profile.to_notebook_iframe()
else:
    print("⚠️ ydata-profilingがインストールされていないため、スキップします")
    print("   インストール: pip install ydata-profiling")


### 手動EDA: 詳細な分析が必要な場合

自動EDAで不十分な場合、以下のセルで詳細な分析を行えます。


In [ ]:
# 詳細なEDA（必要に応じて実行）

# 数値特徴量とカテゴリ特徴量を分離
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()

if 'Id' in numeric_cols:
    numeric_cols.remove('Id')
if 'SalePrice' in numeric_cols:
    numeric_cols.remove('SalePrice')

print(f"数値特徴量: {len(numeric_cols)}個")
print(f"カテゴリ特徴量: {len(categorical_cols)}個")


In [ ]:
# 訓練データとテストデータの分布比較（簡潔版）
if len(numeric_cols) > 0:
    # 相関が高い特徴量を選択
    correlations = train_df[numeric_cols + [target_col]].corr()[target_col].abs().sort_values(ascending=False)
    top_features = correlations.head(3).index.tolist()
    if target_col in top_features:
        top_features.remove(target_col)
    
    # 可視化
    fig, axes = plt.subplots(1, len(top_features), figsize=(5*len(top_features), 5))
    if len(top_features) == 1:
        axes = [axes]
    
    for i, feature in enumerate(top_features):
        axes[i].hist(train_df[feature].dropna(), bins=30, alpha=0.5, label='訓練', density=True)
        axes[i].hist(test_df[feature].dropna(), bins=30, alpha=0.5, label='テスト', density=True)
        axes[i].set_title(feature)
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("💡 分布の違いが大きい場合、モデルの性能に影響する可能性があります")


In [ ]:
# 外れ値（Outlier）の検出
print("=" * 60)
print("📊 外れ値の検出")
print("=" * 60)

if 'SalePrice' in train_df.columns:
    # 目的変数の外れ値を検出（IQR法）
    Q1 = train_df['SalePrice'].quantile(0.25)
    Q3 = train_df['SalePrice'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_df[(train_df['SalePrice'] < lower_bound) | (train_df['SalePrice'] > upper_bound)]
    
    print(f"外れ値の定義:")
    print(f"  下限: ${lower_bound:,.2f}")
    print(f"  上限: ${upper_bound:,.2f}")
    print(f"  外れ値の数: {len(outliers)} ({len(outliers)/len(train_df)*100:.2f}%)")
    
    # 可視化
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # 箱ひげ図
    axes[0].boxplot(train_df['SalePrice'])
    axes[0].set_title('住宅価格の箱ひげ図（外れ値検出）')
    axes[0].set_ylabel('価格（ドル）')
    
    # 散布図（ID vs 価格）
    axes[1].scatter(train_df.index, train_df['SalePrice'], alpha=0.5)
    axes[1].axhline(y=upper_bound, color='r', linestyle='--', label=f'上限: ${upper_bound:,.0f}')
    axes[1].axhline(y=lower_bound, color='r', linestyle='--', label=f'下限: ${lower_bound:,.0f}')
    axes[1].set_title('住宅価格の散布図（外れ値検出）')
    axes[1].set_xlabel('サンプル番号')
    axes[1].set_ylabel('価格（ドル）')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    if len(outliers) > 0:
        print(f"\n⚠️ 外れ値の統計情報:")
        print(outliers['SalePrice'].describe())
else:
    print("⚠️ 目的変数が見つかりませんでした")


In [ ]:
# 重要な数値特徴量と目的変数の散布図
print("=" * 60)
print("📊 重要な数値特徴量と目的変数の関係")
print("=" * 60)

if 'SalePrice' in train_df.columns and len(numeric_cols) > 0:
    # 相関が高い特徴量を選択
    correlations = train_df[numeric_cols + ['SalePrice']].corr()['SalePrice'].abs().sort_values(ascending=False)
    top_features = correlations.head(6).index.tolist()  # 上位6個
    
    if 'SalePrice' in top_features:
        top_features.remove('SalePrice')
    
    # 散布図を描画
    n_features = min(6, len(top_features))
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for i, feature in enumerate(top_features[:6]):
        axes[i].scatter(train_df[feature], train_df['SalePrice'], alpha=0.5)
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('SalePrice')
        axes[i].set_title(f'{feature} vs SalePrice\n(相関: {correlations[feature]:.3f})')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n💡 表示した特徴量: {top_features[:6]}")
    print("   これらの特徴量は価格予測に重要です")
else:
    print("⚠️ 目的変数または数値特徴量が見つかりませんでした")


In [ ]:
# 相関行列の可視化（重要な特徴量のみ）
print("=" * 60)
print("📊 特徴量間の相関行列（重要な特徴量のみ）")
print("=" * 60)

if 'SalePrice' in train_df.columns and len(numeric_cols) > 0:
    # 相関が高い特徴量を選択
    correlations = train_df[numeric_cols + ['SalePrice']].corr()['SalePrice'].abs().sort_values(ascending=False)
    top_features = correlations.head(15).index.tolist()  # 上位15個
    
    # 相関行列を計算
    corr_matrix = train_df[top_features].corr()
    
    # ヒートマップで可視化
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('特徴量間の相関行列（重要な特徴量のみ）')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 相関行列の見方:")
    print("  - 1に近い（赤）: 強い正の相関")
    print("  - -1に近い（青）: 強い負の相関")
    print("  - 0に近い（白）: 相関が弱い")
    print("  - 相関が高い特徴量の組み合わせは、多重共線性の可能性があります")
else:
    print("⚠️ 目的変数または数値特徴量が見つかりませんでした")


In [ ]:
# 訓練データとテストデータの分布の比較
print("=" * 60)
print("📊 訓練データとテストデータの分布比較")
print("=" * 60)

if len(numeric_cols) > 0:
    # 重要な数値特徴量を選択
    correlations = train_df[numeric_cols + ['SalePrice']].corr()['SalePrice'].abs().sort_values(ascending=False)
    top_features = correlations.head(5).index.tolist()
    
    if 'SalePrice' in top_features:
        top_features.remove('SalePrice')
    
    # 訓練データとテストデータの平均値を比較
    comparison = pd.DataFrame({
        '訓練データの平均': train_df[top_features].mean(),
        'テストデータの平均': test_df[top_features].mean()
    })
    comparison['差'] = comparison['テストデータの平均'] - comparison['訓練データの平均']
    comparison['差の割合(%)'] = (comparison['差'] / comparison['訓練データの平均']) * 100
    
    print("\n重要な特徴量の平均値の比較:")
    print(comparison)
    
    # 可視化
    fig, axes = plt.subplots(1, len(top_features), figsize=(5*len(top_features), 5))
    if len(top_features) == 1:
        axes = [axes]
    
    for i, feature in enumerate(top_features):
        axes[i].hist(train_df[feature].dropna(), bins=30, alpha=0.5, label='訓練データ', density=True)
        axes[i].hist(test_df[feature].dropna(), bins=30, alpha=0.5, label='テストデータ', density=True)
        axes[i].set_title(f'{feature}\n訓練平均: {train_df[feature].mean():.1f}\nテスト平均: {test_df[feature].mean():.1f}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('密度')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 分布の違いが大きい場合、モデルの性能に影響する可能性があります")
else:
    print("⚠️ 数値特徴量が見つかりませんでした")


### EDAのまとめ

これまでのEDAで分かったことをまとめます。


In [ ]:
# EDAのまとめ
print("=" * 60)
print("📋 EDAのまとめ")
print("=" * 60)

# 欠損値の再計算（変数が定義されていない場合に備えて）
missing = train_df.isnull().sum()
missing_df_summary = missing[missing > 0]

summary = {
    'データサイズ': f"{train_df.shape[0]}行 × {train_df.shape[1]}列",
    '数値特徴量': f"{len(numeric_cols)}個" if 'numeric_cols' in locals() else "未計算",
    'カテゴリ特徴量': f"{len(categorical_cols)}個" if 'categorical_cols' in locals() else "未計算",
    '欠損値がある列': f"{len(missing_df_summary)}列"
}

if 'SalePrice' in train_df.columns:
    summary['目的変数の範囲'] = f"${train_df['SalePrice'].min():,.0f} - ${train_df['SalePrice'].max():,.0f}"
    summary['目的変数の平均'] = f"${train_df['SalePrice'].mean():,.0f}"
    summary['目的変数の歪度'] = f"{train_df['SalePrice'].skew():.4f}"

print("\n".join([f"  {k}: {v}" for k, v in summary.items()]))

print("\n💡 次のステップ:")
print("  1. 欠損値の処理")
print("  2. 外れ値の処理（必要に応じて）")
print("  3. 特徴量エンジニアリング")
print("  4. カテゴリ変数のエンコーディング")
print("  5. 目的変数の変換（対数変換など）")


## ステップ4: 特徴量エンジニアリング（重要！）

House Pricesコンペでは、**特徴量エンジニアリング**がスコア向上の鍵です。

### よく使われる特徴量エンジニアリング

1. **数値特徴量の組み合わせ**
   - 総面積 = 1階の面積 + 2階の面積
   - 部屋数比率 = 部屋数 / 総面積

2. **カテゴリ特徴量の処理**
   - Label Encoding
   - One-Hot Encoding

3. **目的変数の変換**
   - 対数変換（価格が歪んでいる場合）


In [ ]:
# 前処理の準備
# 訓練データとテストデータを結合して、同じ前処理を適用

# ID列を保存
train_ids = train_df['Id'].copy() if 'Id' in train_df.columns else train_df.index
test_ids = test_df['Id'].copy() if 'Id' in test_df.columns else test_df.index

# 目的変数を保存
y_train = train_df[target_col].copy() if target_col in train_df.columns else None

# 訓練データとテストデータを結合（前処理を統一するため）
if target_col in train_df.columns:
    test_df_temp = test_df.copy()
    test_df_temp[target_col] = -1  # ダミー値
    combined_df = pd.concat([train_df, test_df_temp], axis=0, ignore_index=True)
else:
    combined_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

print(f"結合後のデータサイズ: {combined_df.shape}")
print(f"訓練データ: {len(train_df)}行")
print(f"テストデータ: {len(test_df)}行")


In [ ]:
# 特徴量エンジニアリング
print("=" * 60)
print("🔧 特徴量エンジニアリング")
print("=" * 60)

# 例1: 総面積の作成（存在する場合）
if 'TotalBsmtSF' in combined_df.columns and 'GrLivArea' in combined_df.columns:
    combined_df['TotalSF'] = combined_df['TotalBsmtSF'] + combined_df['GrLivArea']
    print("✅ 総面積（TotalSF）を作成しました")

# 例2: 部屋数と面積の比率（存在する場合）
if 'TotRmsAbvGrd' in combined_df.columns and 'GrLivArea' in combined_df.columns:
    combined_df['RoomsPerArea'] = combined_df['TotRmsAbvGrd'] / (combined_df['GrLivArea'] + 1)
    print("✅ 部屋数比率（RoomsPerArea）を作成しました")

# 例3: 築年数とリノベーション年（存在する場合）
if 'YearBuilt' in combined_df.columns and 'YearRemodAdd' in combined_df.columns:
    combined_df['Age'] = 2024 - combined_df['YearBuilt']
    combined_df['RemodAge'] = 2024 - combined_df['YearRemodAdd']
    print("✅ 築年数（Age）とリノベーション年（RemodAge）を作成しました")

print("\n✅ 特徴量エンジニアリング完了")


In [ ]:
# 欠損値の処理
print("=" * 60)
print("🔧 欠損値の処理")
print("=" * 60)

# 数値列の欠損値を中央値で埋める
numeric_cols = combined_df.select_dtypes(include=[np.number]).columns.tolist()
if 'Id' in numeric_cols:
    numeric_cols.remove('Id')
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

if len(numeric_cols) > 0:
    missing_numeric = [col for col in numeric_cols if combined_df[col].isnull().sum() > 0]
    if len(missing_numeric) > 0:
        print(f"数値列の欠損値を中央値で埋めます: {len(missing_numeric)}列")
        imputer_numeric = SimpleImputer(strategy='median')
        combined_df[numeric_cols] = imputer_numeric.fit_transform(combined_df[numeric_cols])
        print("✅ 数値列の欠損値処理完了")
    else:
        print("✅ 数値列に欠損値はありません")

# カテゴリ列の欠損値を最頻値で埋める
categorical_cols = combined_df.select_dtypes(include=['object']).columns.tolist()
if len(categorical_cols) > 0:
    missing_categorical = [col for col in categorical_cols if combined_df[col].isnull().sum() > 0]
    if len(missing_categorical) > 0:
        print(f"カテゴリ列の欠損値を最頻値で埋めます: {len(missing_categorical)}列")
        imputer_categorical = SimpleImputer(strategy='most_frequent')
        combined_df[categorical_cols] = imputer_categorical.fit_transform(combined_df[categorical_cols])
        print("✅ カテゴリ列の欠損値処理完了")
    else:
        print("✅ カテゴリ列に欠損値はありません")


In [ ]:
# カテゴリデータの数値化（Label Encoding）
print("=" * 60)
print("🔧 カテゴリデータの数値化")
print("=" * 60)

label_encoders = {}

categorical_cols_to_encode = combined_df.select_dtypes(include=['object']).columns.tolist()

if len(categorical_cols_to_encode) > 0:
    print(f"カテゴリ列を数値化します: {len(categorical_cols_to_encode)}列")
    for col in categorical_cols_to_encode:
        le = LabelEncoder()
        combined_df[col] = le.fit_transform(combined_df[col].astype(str))
        label_encoders[col] = le
    print("✅ カテゴリデータの数値化完了")
else:
    print("✅ カテゴリデータはありません")


In [ ]:
# 訓練データとテストデータを再度分離
train_processed = combined_df[:len(train_df)].copy()
test_processed = combined_df[len(train_df):].copy()

# 目的変数を復元
if y_train is not None:
    train_processed[target_col] = y_train

# ID列と目的変数を除外して特徴量のみにする
exclude_cols = ['Id']
if target_col in train_processed.columns:
    exclude_cols.append(target_col)

feature_cols = [col for col in train_processed.columns if col not in exclude_cols]

X_train = train_processed[feature_cols].copy()
y_train_final = train_processed[target_col].copy() if target_col in train_processed.columns else None
X_test = test_processed[feature_cols].copy()

print(f"✅ 前処理完了")
print(f"訓練データの特徴量数: {X_train.shape[1]}")
print(f"訓練データのサンプル数: {X_train.shape[0]}")
print(f"テストデータのサンプル数: {X_test.shape[0]}")

# 目的変数の対数変換（歪度が大きい場合）
if y_train_final is not None:
    if y_train_final.skew() > 0.5:  # 歪度が0.5より大きい場合
        print(f"\n📊 目的変数の対数変換を適用します（歪度: {y_train_final.skew():.4f}）")
        y_train_final_log = np.log1p(y_train_final)  # log1p = log(1+x)で0を扱いやすい
        use_log_transform = True
    else:
        y_train_final_log = y_train_final.copy()
        use_log_transform = False
        print(f"\n📊 目的変数の対数変換は不要です（歪度: {y_train_final.skew():.4f}）")


## ステップ5: モデルの訓練

回帰問題では、以下のモデルがよく使われます：

1. **線形回帰（Linear Regression）**
   - シンプルで解釈しやすい
   - ベースラインとして使われる

2. **Ridge / Lasso回帰**
   - 正則化を加えた線形回帰
   - 過学習を防ぐ

3. **ランダムフォレスト回帰（Random Forest Regressor）**
   - 非線形関係も学習できる
   - 特徴量の重要度が分かる

4. **勾配ブースティング回帰（Gradient Boosting Regressor）**
   - 高い精度が出やすい
   - XGBoost、LightGBMなどが有名


In [ ]:
# モデル1: 線形回帰（ベースライン）
print("=" * 60)
print("🤖 モデル1: 線形回帰")
print("=" * 60)

if y_train_final is not None:
    # データのスケーリング
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 対数変換した目的変数を使う
    y_train_model = y_train_final_log if use_log_transform else y_train_final
    
    # モデルの訓練
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_train_model)
    
    # 訓練データでの予測
    y_train_pred_lr = lr_model.predict(X_train_scaled)
    
    # 対数変換を元に戻す
    if use_log_transform:
        y_train_pred_lr = np.expm1(y_train_pred_lr)  # expm1 = exp(x) - 1
    
    # RMSEを計算
    train_rmse_lr = np.sqrt(mean_squared_error(y_train_final, y_train_pred_lr))
    train_mae_lr = mean_absolute_error(y_train_final, y_train_pred_lr)
    
    print(f"訓練データでのRMSE: ${train_rmse_lr:,.2f}")
    print(f"訓練データでのMAE: ${train_mae_lr:,.2f}")
    
    # テストデータでの予測
    y_test_pred_lr = lr_model.predict(X_test_scaled)
    if use_log_transform:
        y_test_pred_lr = np.expm1(y_test_pred_lr)
    
    print("✅ 線形回帰の訓練完了")
else:
    print("⚠️ 目的変数が見つかりませんでした")


In [ ]:
# モデル2: ランダムフォレスト回帰
print("=" * 60)
print("🤖 モデル2: ランダムフォレスト回帰")
print("=" * 60)

if y_train_final is not None:
    y_train_model = y_train_final_log if use_log_transform else y_train_final
    
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    
    rf_model.fit(X_train, y_train_model)
    
    # 訓練データでの予測
    y_train_pred_rf = rf_model.predict(X_train)
    if use_log_transform:
        y_train_pred_rf = np.expm1(y_train_pred_rf)
    
    train_rmse_rf = np.sqrt(mean_squared_error(y_train_final, y_train_pred_rf))
    train_mae_rf = mean_absolute_error(y_train_final, y_train_pred_rf)
    
    print(f"訓練データでのRMSE: ${train_rmse_rf:,.2f}")
    print(f"訓練データでのMAE: ${train_mae_rf:,.2f}")
    
    # 特徴量の重要度を表示（上位10個）
    feature_importance = pd.DataFrame({
        '特徴量': X_train.columns,
        '重要度': rf_model.feature_importances_
    }).sort_values('重要度', ascending=False)
    
    print("\n📊 特徴量の重要度（上位10個）:")
    print(feature_importance.head(10))
    
    # テストデータでの予測
    y_test_pred_rf = rf_model.predict(X_test)
    if use_log_transform:
        y_test_pred_rf = np.expm1(y_test_pred_rf)
    
    print("\n✅ ランダムフォレスト回帰の訓練完了")
else:
    print("⚠️ 目的変数が見つかりませんでした")


In [ ]:
# モデル3: 勾配ブースティング回帰
print("=" * 60)
print("🤖 モデル3: 勾配ブースティング回帰")
print("=" * 60)

if y_train_final is not None:
    y_train_model = y_train_final_log if use_log_transform else y_train_final
    
    gb_model = GradientBoostingRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    
    gb_model.fit(X_train, y_train_model)
    
    # 訓練データでの予測
    y_train_pred_gb = gb_model.predict(X_train)
    if use_log_transform:
        y_train_pred_gb = np.expm1(y_train_pred_gb)
    
    train_rmse_gb = np.sqrt(mean_squared_error(y_train_final, y_train_pred_gb))
    train_mae_gb = mean_absolute_error(y_train_final, y_train_pred_gb)
    
    print(f"訓練データでのRMSE: ${train_rmse_gb:,.2f}")
    print(f"訓練データでのMAE: ${train_mae_gb:,.2f}")
    
    # テストデータでの予測
    y_test_pred_gb = gb_model.predict(X_test)
    if use_log_transform:
        y_test_pred_gb = np.expm1(y_test_pred_gb)
    
    print("✅ 勾配ブースティング回帰の訓練完了")
    
    # モデルの比較
    print("\n" + "=" * 60)
    print("📊 モデル比較（訓練データでのRMSE）")
    print("=" * 60)
    try:
        print(f"線形回帰: ${train_rmse_lr:,.2f}")
    except NameError:
        print("線形回帰: 未実行")
    try:
        print(f"ランダムフォレスト: ${train_rmse_rf:,.2f}")
    except NameError:
        print("ランダムフォレスト: 未実行")
    print(f"勾配ブースティング: ${train_rmse_gb:,.2f}")
else:
    print("⚠️ 目的変数が見つかりませんでした")


## ステップ6: 予測と提出ファイルの作成

回帰問題では、予測値が負の値にならないように注意が必要です。
（住宅価格が負の値になることはないため）


In [ ]:
# 提出ファイルの作成
print("=" * 60)
print("📝 提出ファイルの作成")
print("=" * 60)

# 最も良い性能を示したモデルを使う
try:
    best_predictions = y_test_pred_gb.copy()
    print("✅ 勾配ブースティング回帰の予測を使用")
except NameError:
    try:
        best_predictions = y_test_pred_rf.copy()
        print("✅ ランダムフォレスト回帰の予測を使用")
    except NameError:
        try:
            best_predictions = y_test_pred_lr.copy()
            print("✅ 線形回帰の予測を使用")
        except NameError:
            print("⚠️ エラー: 予測結果が見つかりませんでした")
            best_predictions = None

if best_predictions is not None:
    # 負の値を0に変換（住宅価格が負になることはない）
    best_predictions = np.maximum(best_predictions, 0)
    
    # 提出ファイルを作成
    submission = pd.DataFrame({
        'Id': test_ids,
        'SalePrice': best_predictions
    })
    
    # ファイルを保存
    submission_file = '../submission_house_prices.csv'
    submission.to_csv(submission_file, index=False)
    
    print(f"✅ 提出ファイルを作成しました: {submission_file}")
    print(f"\n提出ファイルの最初の10行:")
    print(submission.head(10))
    print(f"\n提出ファイルの統計情報:")
    print(submission['SalePrice'].describe())
else:
    print("⚠️ 提出ファイルを作成できませんでした")


## 📚 次のステップ（改善のヒント）

### 1. より高度な特徴量エンジニアリング
- カテゴリ変数の組み合わせ
- ポリノミアル特徴量（特徴量の2乗、3乗など）
- ターゲットエンコーディング

### 2. モデルのチューニング
- ハイパーパラメータの最適化
- グリッドサーチやランダムサーチ

### 3. より高度なモデル
- XGBoost、LightGBM、CatBoost
- スタッキング（複数モデルの組み合わせ）

### 4. 交差検証の活用
- K-Fold交差検証でより正確な性能評価
- 過学習を防ぐ

## 🎉 お疲れ様でした！

このノートブックで回帰問題の基本的な流れを学びました。
次は、上記の改善方法を試して、スコアを向上させてみましょう！
